In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
# import simpreglib as srl
import pandas as pd
import numpy as np
import datetime as dt

# for linear regression models
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import scipy.stats as stats

# run_RFE and run_RFECV
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression


from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings # weird sns.distplot warnings
warnings.filterwarnings("ignore")

plt.style.use('fivethirtyeight')

In [2]:
# creates dictionary of variance inflation factors. 
def create_vif_dictionary(X):

    vif_dict = {}
    for i in range(len(X.columns)):
        vif = variance_inflation_factor(X.values, i)
        v = X.columns[i]
        vif_dict[v] = vif

    good_vifs = []
    bad_vifs = []

    for k,v in vif_dict.items():
        if v < 10:
            good_vifs.append(k)
        else:
            bad_vifs.append(k)

    return good_vifs,bad_vifs

# create a dictionary showing the adjusted R-squared values for each feature individually
def create_R2_dictionary(X,y):
    adj_R_squares = {}
    for feature in X.columns:
        predictors_int = sm.add_constant(X[feature])
        model = sm.OLS(y,predictors_int).fit()
        adj_R_square = float(model.summary2().tables[0][3][0])
        adj_R_squares[feature] = adj_R_square
    return adj_R_squares

# author's docstring is in a markup cell down below in the stepwise selection section
def stepwise_selection(X, y, 


                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))
        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            worst_feature_name = included[worst_feature]
            included.pop(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature_name, worst_pval))
        if not changed:
            break
    return included

def run_RFE(X,y):
    linreg = LinearRegression()
    selector = RFE(linreg)
    selector = selector.fit(X, y)

    selected = selector.support_ # list of bools representing if feature is selected or not
    selections = [] # list of column names that are selected
    labels = list(X.columns) # list of all column names 

    for idx,feature in enumerate(selected): # append labels of selected features to selections list
        if feature == True:
            selections.append(labels[idx])
        else:
            pass
    
    ranked = selector.ranking_ # list of bools representing if feature is selected or not
    rankers = [] # list of column names that are selected
    labels = list(X.columns) # list of all column names 

    for idx,feature in enumerate(ranked): # append labels of selected features to selections list
        if feature == 1:
            rankers.append(labels[idx])
        else:
            pass
    return selections, rankers

def run_RFECV(X,y):
    linreg = LinearRegression()
    selector = RFECV(linreg)
    selector = selector.fit(X, y)

    selected = selector.support_ # list of bools representing if feature is selected or not
    selections = [] # list of column names that are selected
    labels = list(X.columns) # list of all column names 

    for idx,feature in enumerate(selected): # append labels of selected features to selections list
        if feature == True:
            selections.append(labels[idx])
        else:
            pass
    
    ranked = selector.ranking_ # list of bools representing if feature is selected or not
    rankers = [] # list of column names that are selected
    labels = list(X.columns) # list of all column names 

    for idx,feature in enumerate(ranked): # append labels of selected features to selections list
        if feature == 1:
            rankers.append(labels[idx])
        else:
            pass

    return selections, rankers

def run_OLS_model(X,y):


    predictors_int = sm.add_constant(X)
    model = sm.OLS(y,predictors_int).fit()
    return model




Set up predictor and target dataframes, deal with unusable data

In [3]:
X = pd.read_csv('data/kc_house_data.csv')
X.drop(X.loc[X['sqft_basement']=='?'].index,inplace=True) # remove '?' from the data.
y = X.price
y_log = np.log(y) # target vairable log transformed for normality
X.drop(['price','id','lat','long'],inplace=True,axis=1)

Engineer data to present as numerical values, not objects. 

In [4]:
# convert all string types into np floats
X.sqft_basement = [float(sq) for sq in list(X.sqft_basement)]

# Replaces grade strings with numerics based on data dict. 
grade_raws = list(X.grade.unique())
# replaces a cell value with the int of the first character of its existing string
for raw in grade_raws:
    X.grade.replace(to_replace=raw,value=int(raw[0]),inplace=True)

# replaces condition objects with numerics based on data dict.
condition_dict = {'Poor':1,'Fair':2,'Average':3,'Good':4,'Very Good':5}
for key in condition_dict:
    X.condition.replace(to_replace=condition_dict,inplace=True)

# replace yr_built NaNs with numeric 0
X.yr_renovated.replace(to_replace=np.nan,value=0,inplace=True)

In [5]:
# convert waterfront into numeric boolean
waterfront_bool_list = []
for value in X.waterfront:
    if value == 'YES':
        waterfront_bool_list.append(1)
    else:
        waterfront_bool_list.append(0)
X.waterfront = waterfront_bool_list

In [6]:
# convert view from string into categorical ordinal
view_rank_list = [] 
view_dict = {'NONE':0,'FAIR':1,'AVERAGE':2,'GOOD':3,'EXCELLENT':4}

for value in X.view:
    if value in list(view_dict.keys()):
        view_rank_list.append(view_dict[value])
    else:
        view_rank_list.append(0)
        
X.view = view_rank_list

Engineer date into a usable type of number (datetime ordinal)

In [7]:
# convert dates into ordinals
X.date = pd.to_datetime(X['date'])
X.date = X['date'].map(dt.datetime.toordinal)

In [8]:
null_dict = {} # creates dictionary where keys are the labels and values are the count of null values
for feature in X.columns:
    null_count = sum(X[feature].isna())
    null_dict[feature] = null_count
# null_dict

Investigate multicolinearity before engineering inferred features

In [9]:
# X.corr()
cor_df=X.corr().abs().stack().reset_index().sort_values(0, ascending=False)
cor_df['pairs'] = list(zip(cor_df.level_0, cor_df.level_1))
cor_df.set_index(['pairs'], inplace = True)
cor_df.drop(columns=['level_1', 'level_0'], inplace = True)
cor_df.columns = ['cc']
cor_df.drop_duplicates(inplace=True)

In [10]:
# cor_df[(cor_df.cc>.60) & (cor_df.cc <1)]

In [11]:
# colinear_drops = ['sqft_above','sqft_living15','bathrooms','sqft_lot15']

Engineering inferred feature
- bed bath ratio:           ratio of bedrooms to bathrooms
- level ratios:             ratio of square feet above 'grade' and below (ratio of everything else to the basement)
- live_lot_ratio:           ratio of living space to lot size
- relative living space:    ratio of living space to the living space of the nearest 15 houses (sqft_living :: sqft_living15)
- relatve lot size:         same as living space but for lot size instead. 
- level difference:         difference in square footage of living space to basement space

In [12]:
bbratios = []
lvl_ratios = []
live_lot_ratio = []

for index,row in X.iterrows(): #iterate through every record
#     print(row)
    bbratio = row.bedrooms/row.bathrooms # calculate ratio of bedrooms to bathrooms
    bbratios.append(bbratio) # append ratio to the list

    LLratio = row.sqft_living/row.sqft_lot
    live_lot_ratio.append(LLratio)

    if row.sqft_basement == 0: # sqft_basement is zero if there is no basement
        lvl_ratios.append(0) # ratio should also be zero if there is no ratio
    else:
        lvl_ratio = row.sqft_above / row.sqft_basement # calculate ratio of space above grade vs below grade
        lvl_ratios.append(lvl_ratio) # append ratio to the list

X['bed_bath_ratio'] = bbratios # create new column and asign list as its values
X['level_ratio'] = lvl_ratios # create new column and asign list as its values
X['live_lot_ratio'] = live_lot_ratio # you get the idea . . . 

In [13]:
rel_live_space = []
rel_lot_size = [] 
rel_difference = []

for index,row in X.iterrows(): # for every record 
    live_dif = row.sqft_living - row.sqft_living15 # calculate difference in sqft of the given house and the nearest 15 other houses
    rel_live_space.append(live_dif) # append it to the list

    lot_dif = row.sqft_lot - row.sqft_lot15 # calculate difference in sqft of the given lot and the nearest 15 other lots
    rel_lot_size.append(lot_dif) # append it to the list

    lvl_dif = row.sqft_above - row.sqft_basement # calculate difference between space above grade and below grade
    rel_difference.append(lvl_dif) # append it to the list

X['relative_living_space'] = rel_live_space # assign respective list to new column 
X['relative_lot_size'] = rel_lot_size
X['level_difference'] = rel_difference

In [14]:
water_loc_dict = {'Duwamish':[98168],
'Elliott Bay':[98119,98104,98129,98132,98127,98125,98195,98101,98134,98170,98139,98131,98181], 
'Puget Sound':[98071,98083,98013,98070,98031,98131,98063,98195,98207,98190], 
'Lake Union':[98109], 
'Ship Canal':[00000], 
'Lake Washington':[98072,98077], 
'Lake Sammamish':[98074,98075,98029], 
'other lake':[00000], 
'river/slough waterfronts':[00000]}

waterfront_list = []

for zip in X.zipcode:
    for k,v in water_loc_dict.items():
        if zip in v:
            waterfront_list.append(k)
            appended = True
            break
        else: 
            appended = False
    if not appended:
        waterfront_list.append('NONE')

# print(len(waterfront_list),set(waterfront_list))
X['waterfront_loc'] = waterfront_list

In [15]:
waterfront_dummies = pd.get_dummies(X.waterfront_loc).drop('NONE',axis=1).add_prefix('waterfront_')

X = X.drop(['waterfront_loc','zipcode'],axis=1)
hot_water = pd.concat([X,waterfront_dummies],axis=1)

In [17]:
# hot_water.info()

In [19]:
cor_df=hot_water.corr().abs().stack().reset_index().sort_values(0, ascending=False)
cor_df.head()
# cor_df['pairs'] = list(zip(cor_df.level_0, cor_df.level_1))
# cor_df.set_index(['pairs'], inplace = True)
# cor_df.drop(columns=['level_1', 'level_0'], inplace = True)
# cor_df.columns = ['cc']
# cor_df.drop_duplicates(inplace=True)

,level_0,level_1,0
0,date,date,1.0
406,sqft_living15,sqft_living15,1.0
87,sqft_living,sqft_living,1.0
116,sqft_lot,sqft_lot,1.0
145,floors,floors,1.0


In [ ]:
# cor_df[(cor_df.cc>.60) & (cor_df.cc <1)]

# is it a good decision to drop sqft_living in lieu of relative_living_space ???

In [ ]:
# columns to drop based one colinearity
co_drops = ['sqft_living','sqft_above','sqft_living15','bathrooms','sqft_lot15','sqft_lot','level_ratio'] # included level ratio since its basically same as level difference

In [ ]:
Xd = X.drop(co_drops,axis=1)
# Xd.columns

- investigage mapping zipcode, or coordinates to view ? Not enough coordinate info... 
- consider dropping zip/lot/long and then re running notebook from the top, (i.e. their presence may affect colinearity of other features?) 
    - did not affect colinearity but it cleaned up some clutter

##### Linear Transformation and Feature ranking 
using standardization (x - x_bar / sigma) to normalize units of measurement

investigate feature rankings using:
- stepwise method
- recursive elimination


In [ ]:
Xs = pd.DataFrame([]) # data standardized
for feature in Xd.columns:
    x_comp = (Xd[feature] - np.mean(Xd[feature])/np.std(Xd[feature])) # (x - x_bar)/sigma
    Xs[feature] = x_comp 

In [ ]:
Xlog = pd.DataFrame([]) # standardized data log transformed
for feature in Xs.columns:
    x_logged = np.log(Xs[feature])
    Xlog[feature] = x_logged

# dealing with NaN values resulting from the log transformation
for column in Xlog:
    Xlog[column].replace(np.NaN,0,inplace=True)

Create a baseline model for refrence

In [ ]:
y_log_base = run_OLS_model(Xd,y_log)
y_log_pred_stand = run_OLS_model(Xs,y_log)
# y_log_pred_log = run_OLS_model(Xlog,y_log)


In [ ]:
y_log_base.summary()

In [ ]:
# Q-Q plots 
# for idx,feature in enumerate(Xs.columns):
#     logged = np.log(Xs[feature])
#     sm.qqplot(logged, line ='45')
#     plt.title(feature)

# plt.tight_layout()
# plt.show()

Stepwise selection method

In [ ]:
stepwisers = stepwise_selection(Xd, y_log, Xd.columns,threshold_in=0.01, threshold_out = 0.05, verbose=False)
stepwised_df = Xd[stepwisers]

# appears same as baseline 
stepped_model = run_OLS_model(stepwised_df,y_log)
stepped_model.summary()

In [ ]:
# one hot encode grade
# inspect multicolinearity once more

In [ ]:
# from sklearn.preprocessing import LabelBinarizer

# lb = LabelBinarizer()
# condition_dummies = lb.fit_transform(stepwised_df.condition)
# # You need to convert this back to a dataframe
# con_dum_df = pd.DataFrame(condition_dummies,columns=lb.classes_).add_prefix('condition_')

# step_coded = pd.concat([stepwised_df,con_dum_df],axis=1)

In [ ]:
# cc_df=step_coded.corr().abs().stack().reset_index().sort_values(0, ascending=False)
# cc_df['pairs'] = list(zip(cc_df.level_0, cc_df.level_1))
# cc_df.set_index(['pairs'], inplace = True)
# cc_df.drop(columns=['level_1', 'level_0'], inplace = True)
# cc_df.columns = ['cc']
# cc_df.drop_duplicates(inplace=True)

# cc_df[(cc_df.cc>.60) & (cc_df.cc <1)] # drop condition_4, condition_3

In [ ]:
# step_coded = pd.concat([y_log,step_coded],axis=1)
# step_coded = step_coded.drop(['condition_4','condition_3','condition_1','condition_2','condition_5'],axis=1)
# step_coded = step_coded.dropna()


In [ ]:
Nanf_dict = {}

for feature in hot_stepped.columns:
    Nanf_dict[feature] = sum(hot_stepped[feature].isna())

Nanf_dict


In [ ]:
hot_step_model = run_OLS_model(hot_stepped,y_log)
hot_step_model.summary()

Recursive elimation: with and without cross-validation
- does not produce better results than stepwise selection

In [ ]:
selected, ranking = run_RFECV(Xd,y_log)
RFECV_df = Xd[selected]

# also identtical to baseline 
rfe_model = run_OLS_model(RFECV_df,y_log)
# rfe_model.summary()

Manual selection based on P value and variance inflation factor

In [ ]:
good,bad = create_vif_dictionary(Xd)